<a href="https://colab.research.google.com/github/gurdeep41490/DockerCasts/blob/master/beam_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 13.4 MB/s eta 0:00:00
  

In [2]:
!mkdir -p data

In [3]:
!pip3 install google_colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.9 MB/s eta 0:00:00


In [4]:
from google.colab import files

In [5]:
uploaded = files.upload()

Saving dept_data.txt to dept_data.txt


In [6]:
!ls

data  dept_data.txt  sample_data


In [46]:
import apache_beam as beam

p1 = beam.Pipeline()

def createTuple(record):
  return (record[1], 1)

att_count = (
   p1
    | 'ReadFile' >> beam.io.ReadFromText('dept_data.txt')
    | 'Split it' >> beam.Map(lambda record: record.split(','))
    | 'Filter it' >> beam.Filter(lambda record: record[3] == 'Accounts')
    | 'Form pairs' >> beam.Map(createTuple)
    | 'combine' >> beam.CombinePerKey(sum)
    | 'final write' >> beam.io.WriteToText('data/op_final')

)

p1.run()

!{('head -n 20 data/op_final-00000-of-00001')}

('Marco', 31)
('Rebekah', 31)
('Itoe', 31)
('Edouard', 31)
('Kyle', 62)
('Kumiko', 31)
('Gaston', 31)
('Ayumi', 30)


In [21]:
import apache_beam as beam

class CommonTransoform(beam.PTransform):

  def expand(self, input_collection):
    comm_trsfm = (
        input_collection
         | "create tuple" >> beam.Map(createTuple)
         | "combine" >> beam.CombinePerKey(sum)
    )
    return comm_trsfm


class splitRow(beam.DoFn):

  def process(self, element):
    return [element.split(",")]

def createTuple(record):
  return (record[1], 1)

p1 = beam.Pipeline()

input_collection = (
    p1
    | "Read File" >> beam.io.ReadFromText('dept_data.txt')
    | "separate elements" >> beam.ParDo(splitRow())
    #|"separate element" >> beam.Map(lambda element: element.split(','))
)

acc_coll = (
    input_collection
    | "filter accs" >> beam.Filter(lambda element: element[3] == 'Accounts')
    | "acc transform" >> CommonTransoform()
    | "Write to File" >> beam.io.WriteToText('data/acc_file')
)

hr_coll = (
    input_collection
    | "Filter Hrs" >> beam.Filter(lambda element: element[3] == 'HR')
    | "hr transform" >> CommonTransoform()
    | "WriteToFile" >> beam.io.WriteToText('data/hr_file')
)

p1.run()

In [24]:
!ls data/

acc_file-00000-of-00001
beam-temp-hr_file-397798324fc611efa3da0242ac1c000c
beam-temp-hr_file-a43674cc4fc611efa3da0242ac1c000c
hr_file-00000-of-00001
